In [7]:
import sys
import os

sys.path.append("../")

In [8]:
from src.common_functions import *

# 1. Load data, prep & train test split

In [9]:
df = pd.read_csv(r"../datasets/fin_churn.csv" )
df = df.rename(columns=dict(
    zip(df.columns, list(map(to_snake_case, df.columns)))))
df.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
def check_positive_or_zero(column):
    # Check if all values are positive or 0
    is_positive_or_zero = all(pd.to_numeric(column, errors='coerce') >= 0)
    return is_positive_or_zero

    

In [23]:
result = {col: check_positive_or_zero(df[col]) for col in df.columns}

print(result)

{'row_number': True, 'customer_id': True, 'surname': False, 'credit_score': True, 'geography': False, 'gender': False, 'age': True, 'tenure': True, 'balance': True, 'num_of_products': True, 'has_cr_card': True, 'is_active_member': True, 'estimated_salary': True, 'exited': True}


In [28]:
nan_values = df.isna().any()
print(nan_values)

row_number          False
customer_id         False
surname             False
credit_score        False
geography           False
gender              False
age                 False
tenure              False
balance             False
num_of_products     False
has_cr_card         False
is_active_member    False
estimated_salary    False
exited              False
dtype: bool


In [25]:
y = "exited"

# some generic cleaning

COLUMNS_TO_DROP = ["customer_id"]
assert df[y].isna().any() == False
raw_features = df.drop(COLUMNS_TO_DROP, axis=1)

# ML tuning

train, test = train_test_split(
    raw_features, test_size=0.2, random_state=0, stratify=df[y])

fit_le = LabelEncoder().fit(train[y])

In [27]:
param_grid_log_reg = {
    'logistic__penalty': ['l2'],
    'logistic__C': [1, 10, 100, 1000]
}

model = LogisticRegression()

grid = sklearn_gridsearch_using_pipeline(
    train, y_name=y, model=model, fit_le=fit_le, model_name='logistic', param_grid_model=param_grid_log_reg)
best_pipeline_log_reg = grid.best_estimator_
best_pipeline_log_reg

Fitting 5 folds for each of 28 candidates, totalling 140 fits


c:\Users\stijn\.virtualenvs\End-to-end-project---Customer-churn-eLOWfj3z\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Best score: nan
Best parameters: {'logistic__C': 1, 'logistic__penalty': 'l2', 'pca__n_components': 15}


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['row_number', 'credit_score',
                                                   'age', 'tenure', 'balance',
                                                   'num_of_products',
                                                   'has_cr_card',
                                                   'is_active_member',
                                                   'estimated_salary']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['surname', 'geography',
                                                   'gender'])])),
                ('pca', PCA(n_components=15)),
                ('logistic', LogisticRegression(C=1))])